In [27]:
from delta import  # Delta Lake 라이브러리 임포트 *
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 *
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 *
import timeit

In [2]:
builder = (SparkSession.builder  # SparkSession 빌더 패턴 시작
           .appName("z-order-delta-table")  # 애플리케이션 이름 설정
           .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
           .config("spark.executor.memory", "512m")  # Spark 설정 옵션
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Spark 설정 옵션
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Spark 설정 옵션)

spark = configure_spark_with_delta_pip(builder).getOrCreate()  # SparkSession 생성 또는 기존 세션 반환
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-233a37d2-006e-4ab2-a07f-581a415f3f6f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 296ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0 

In [24]:
%load_ext sparksql_magic
%config SparkSql.limit=20

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [5]:
# 읽기 the CSV file into a Spark DataFrame
df = (spark.read
      .format("csv")
      .option("header", "true")  # 첫 번째 행을 헤더로 사용
      .option("inferSchema", "true")  # 스키마 자동 추론
      .load(  # 파일 로드"../data/Online_Retail.csv"))

# 쓰기 the DataFrame into a Delta table
(df.write
 .format("delta")  # Delta Lake 형식으로 저장
 .mode("overwrite")  # 기존 데이터 덮어쓰기
 .save("../data/delta_lake/online_retail"))

In [31]:
# Query the original table
query = "spark.sql(\"SELECT StockCode, CustomerID, SUM(Quantity) AS TotalQuantity FROM delta.`/opt/workspace/data/delta_lake/online_retail` GROUP BY StockCode, CustomerID\").show()  # DataFrame 내용 출력"
query_time = timeit.timeit(query, number=1, globals=globals())
print(f"Time taken for original table: {query_time} seconds")

+---------+----------+-------------+
|StockCode|CustomerID|TotalQuantity|
+---------+----------+-------------+
|    22637|     15311|           15|
|    22141|     17920|            4|
|    22242|     17920|            5|
|    21257|     14849|            8|
|    21670|     17841|           85|
|   85123A|     15235|           12|
|    21042|     13715|            3|
|    22752|     12471|           11|
|    22726|     13418|           72|
|   85099B|     14388|           70|
|    22094|     18041|           12|
|    21211|     16916|            1|
|    22335|     14449|           15|
|    84347|     15061|         1200|
|    20752|     15574|            1|
|    22593|     16546|         -144|
|    71477|     13295|           -1|
|   85014B|     18239|            3|
|    21916|     15093|           24|
|    79321|     12841|           33|
+---------+----------+-------------+
only showing top 20 rows

Time taken for original table: 5.887500449000072 seconds


In [35]:
# Get the DeltaTable  # Delta 테이블 작업을 위한 클래스 object for the online_retail table
deltaTable = DeltaTable  # Delta 테이블 작업을 위한 클래스.forPath(spark, "/opt/workspace/data/delta_lake/online_retail")

# Optimize the table with Z-Ordering on StockCode and CustomerID
deltaTable.optimize().executeZOrderBy("StockCode", "CustomerID")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [37]:
# Query the Z-Ordered table
query = "spark.sql(\"SELECT StockCode, CustomerID, SUM(Quantity) AS TotalQuantity FROM delta.`/opt/workspace/data/delta_lake/online_retail` GROUP BY StockCode, CustomerID\").show()  # DataFrame 내용 출력"
query_time = timeit.timeit(query, number=1, globals=globals())
print(f"Time taken for z-ordered table: {query_time} seconds")

+---------+----------+-------------+
|StockCode|CustomerID|TotalQuantity|
+---------+----------+-------------+
|    22637|     15311|           15|
|    22141|     17920|            4|
|    22242|     17920|            5|
|    21257|     14849|            8|
|    21670|     17841|           85|
|   85123A|     15235|           12|
|    21042|     13715|            3|
|    22752|     12471|           11|
|    22726|     13418|           72|
|   85099B|     14388|           70|
|    22094|     18041|           12|
|    21211|     16916|            1|
|    22335|     14449|           15|
|    84347|     15061|         1200|
|    20752|     15574|            1|
|    22593|     16546|         -144|
|    71477|     13295|           -1|
|   85014B|     18239|            3|
|    21916|     15093|           24|
|    79321|     12841|           33|
+---------+----------+-------------+
only showing top 20 rows

Time taken for z-ordered table: 1.648877622999862 seconds


In [38]:
spark.stop()  # Spark 세션 종료 - 리소스 정리